# Лабораторная работа №6
*Вариант 2*

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Markdown
import mylib

## Задача 1
Вычислить значение интеграла $I=\int\limits_0^2 P_m (x)dx$, где $P_m(x) = \sum\limits_{i=0}^m c_i x^i$, с помощью квадратурных формул левых прямоугольников и по формуле трапеций с точностью $\varepsilon = 10^{-14}$ . Вычислить интеграл по формуле Гаусса. 

In [4]:
# Условие
coeffs = np.array([1, 0.9, 0.8, 0.7, 0, 2.2, -4.1, 0, -3.4, 3.5])

def get_polynom(c):
    c = np.asarray(c)
    m = len(c)
    def polynom_func(x):
        x = np.asarray(x)
        if x.ndim == 0:
            x_powers = x ** np.arange(0, m)
        else:
            x_powers = x[:, np.newaxis] ** np.arange(0, m)
        return np.dot(x_powers, c)
    return polynom_func

P = get_polynom(coeffs) 
eps = 10**-14
a = 0
b = 2


In [5]:
# Аналитическое решение
coeffs_anti = np.zeros(len(coeffs) + 1, dtype=np.float64)
for i, c in enumerate(coeffs):
    coeffs_anti[i + 1] = np.float64(np.float64(c) / np.float64(i + 1))  # коэффициент при x^(i+1)

antiderivative = get_polynom(coeffs_anti)  # Первообразная
I = antiderivative(b) - antiderivative(a)  # Формула Ньютона Лейбница

# Формируем формулу для отображения
terms = []
for i, c in enumerate(coeffs):
    if i == 0:
        # Для константы: c0 * x
        sign = '-' if c < 0 else ''
        terms.append(f"{sign}{abs(c):.2f}x")
    else:
        # Для старших степеней: c_i/(i+1) * x^(i+1)
        sign = '+' if c >= 0 else '-'
        terms.append(f"{sign}\\frac{{{abs(c):.2f}}}{{{i+1}}}x^{{{i+1}}}")

I_formula = '\\Big(' + ''.join(terms) + '\\Big)' + fr"\Big|_{{{a}}}^{{{b}}}"

display(Math(fr"""
I = \Phi(x) \Big|_{{{a}}}^{{{b}}} = {I_formula} = {I:.10f}
"""))

<IPython.core.display.Math object>

In [6]:
# Вычисление количества точек разбиения для достижения заданной точности
coeffs_prime = coeffs[1:] * np.arange(1, len(coeffs))
P_prime = get_polynom(coeffs_prime)
coeffs_double = coeffs_prime[1:] * np.arange(1, len(coeffs_prime))
P_double = get_polynom(coeffs_double)

x_to_check = np.linspace(a, b, int(1e6))
y_prime = P_prime(x_to_check)
M1, x_M1 = (np.max(y_prime), x_to_check[np.argmax(y_prime)])

y_double = P_double(x_to_check)
M2, x_M2 = (np.max(y_double), x_to_check[np.argmax(y_double)])
M2 = np.max(P_double(x_to_check))

n_left_rect = int(np.ceil((b-a)**2 * M1 / (2 * eps * 1e7)))
n_trap = int(np.ceil(np.sqrt((b-a) ** 3 * M2 / (12 * eps))))




display(Markdown(f"""
Количество равномерно распределённых точек для нахождения максимума: {len(x_to_check):e}  
Максимум M1 достигнут в точке: $x = {x_M1}$  
Максимум M2 достигнут в точке: $x = {x_M2}$
"""))
display(Math(r"""
n_\text{left\_rect} = \Big\lceil \frac{(a + b)^2 M_1}{2 \varepsilon}\Big\rceil
""" + 
fr"= \Big\lceil \frac{{({a} + {b})^2 \cdot {M1}}}{{2 \cdot {eps}}}\Big\rceil" +
f"={n_left_rect:e}"))

display(Math(r"""
n_\text{trap} = \Big\lceil \frac{(a + b)^3 M_2}{12 \varepsilon}\Big\rceil
""" + 
fr"= \Big\lceil \frac{{({a} + {b})^3 \cdot {M2}}}{{12 \cdot {eps}}}\Big\rceil" +
f"={n_trap:e}"))


Количество равномерно распределённых точек для нахождения максимума: 1.000000e+06  
Максимум M1 достигнут в точке: $x = 2.0$  
Максимум M2 достигнут в точке: $x = 2.0$


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
# Формула левых прямоугольников
def left_rectangle_raw(f, a, b, n):
    """
    Прямое использование метода  
    **Очень долгое** для n=1e7: 30 сек.  
    При n=1e6 выполняется за 3 сек.
    """
    h = (b - a) / n
    result = 0
    x = a
    for _ in range(n):
        result += f(x)
        x += h
    
    return result * h

def left_rectangle(f, a, b, n):
    """
    Прямое использование метод с **numpy массивами** 
    Быстрее в 13 раз left_rectangle_raw 
    **Долгое** для n=1e8: 20 сек.
    При n=1e7 выполняется за 2 сек.
    """
    x, h = np.linspace(a, b, n, retstep=True)
    y = f(x)
    return np.sum(y) * h

def left_rectangle_composite(f, a, b, n, chunk_size=1_000_000):
    if n <= chunk_size:
        return left_rectangle(f, a, b, n)
    h = (b - a) / n
    num_chunks = (n + chunk_size - 1) // chunk_size
    actual_chunk_size = n // num_chunks
    remainder = n - actual_chunk_size * num_chunks
    
    result = 0.0
    current_a = a
    
    for i in range(num_chunks):
        current_n = actual_chunk_size + (1 if i < remainder else 0)
        if current_n == 0:
            continue
            
        current_b = current_a + current_n * h
        result += left_rectangle(f, current_a, current_b, current_n)
        current_a = current_b
    
    return result
left_rectangle_composite(P, a, b, int(1e8))    
    



np.float64(122.20647154556569)

In [ ]:
def trapezoidal(f, a, b, n):
    h = (b - a) / n
    x = a
    result = (f(a) + f(b)) / 2  # полусумма крайних точек
    
    for i in range(1, n):
        x += h
        result += f(x)
    
    return result * h
I_trap_e14 = trapezoidal(P, a, b, int(3e7))

In [ ]:
display(mylib.show_large_number(f"{I_trap_e14:.17f}", "das"))
display(mylib.show_large_number(f"{I:.17f}"))
